In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>OCR Camera App</title>
    <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/css/bootstrap.min.css">
    <link rel="stylesheet" href="/static/css/style.css">
    <style>
        body {
            background-color: #d7f3fc; /* Light blue background */
        }
        .navbar-brand img {
            height: 40px; /* Adjust logo size */
        }
    </style>
</head>
<body>
    <!-- Navbar -->
    <nav class="navbar navbar-expand-lg navbar-dark bg-dark">
        <div class="container-fluid">
            <a class="navbar-brand" href="#">
                <img src="/static/images/Black RV Logo.png" alt="Logo"> OCR Camera App
            </a>
        </div>
    </nav>

    <!-- Main Container -->
    <div class="container my-5">
        <div class="text-center">
            <h1 class="mb-4 text-primary">Capture and Extract Text</h1>
            <p class="text-secondary">Easily capture text from images using your camera.</p>
        </div>

        <!-- Camera and Preview Section -->
        <div class="d-flex flex-column align-items-center mb-4">
            <input type="file" accept="image/*" capture="environment" id="cameraInput" style="display:none;">
            <button id="cameraButton" class="btn btn-primary btn-lg mb-3">
                <i class="bi bi-camera"></i> Open Camera
            </button>
            <img id="preview" alt="Captured Image" class="img-thumbnail" style="display:none; max-width: 300px;">
            <button id="uploadButton" class="btn btn-success mt-3" disabled>
                <i class="bi bi-upload"></i> Process Image
            </button>
        </div>

        <!-- Result Section -->
        <div id="result" class="card shadow" style="display:none;">
            <div class="card-header bg-primary text-white">
                <h2 class="h5 mb-0">Results</h2>
            </div>
            <div class="card-body">
                <h3 class="h6 text-secondary">Extracted Text:</h3>
                <pre id="extractedText" class="bg-light p-3 rounded"></pre>
                <h3 class="h6 text-secondary mt-4">Confidence Scores:</h3>
                <ul id="confidenceList" class="list-group list-group-flush"></ul>
                <div class="mt-4 text-center">
                    <h3 class="h6 text-secondary">Annotated Image:</h3>
                    <img id="resultImage" alt="Annotated Image" class="img-thumbnail">
                </div>
            </div>
        </div>
    </div>

    <!-- Bootstrap Bundle JS -->
    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0-alpha3/dist/js/bootstrap.bundle.min.js"></script>
    <!-- Custom JavaScript -->
    <script>
        const cameraButton = document.getElementById('cameraButton');
        const cameraInput = document.getElementById('cameraInput');
        const preview = document.getElementById('preview');
        const uploadButton = document.getElementById('uploadButton');
        const resultContainer = document.getElementById('result');
        const extractedTextElement = document.getElementById('extractedText');
        const confidenceList = document.getElementById('confidenceList');
        const resultImage = document.getElementById('resultImage');

        let capturedImage;

        // Open Camera on Button Click
        cameraButton.addEventListener('click', () => {
            cameraInput.click();
        });

        // On Image Capture
        cameraInput.addEventListener('change', (event) => {
            const file = event.target.files[0];
            if (file) {
                capturedImage = file;
                const reader = new FileReader();
                reader.onload = (e) => {
                    preview.src = e.target.result;
                    preview.style.display = 'block';
                    uploadButton.disabled = false;
                };
                reader.readAsDataURL(file);
            }
        });

        // Process Image after Capture
        uploadButton.addEventListener('click', async () => {
            const formData = new FormData();
            formData.append('image', capturedImage);

            const response = await fetch('/process-image', {
                method: 'POST',
                body: formData
            });

            const result = await response.json();
            extractedTextElement.textContent = result.text.map(item => item.text).join('\n');
            confidenceList.innerHTML = result.text.map(item => `
                <li class="list-group-item">
                    ${(item.confidence *1).toFixed(2)} % accurate
                </li>`).join('');
            resultImage.src = 'data:image/jpeg;base64,' + result.image;
            resultContainer.style.display = 'block';
        });
    </script>
</body>
</html>
